In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sbn

from sklearn.metrics import f1_score
from matplotlib import pyplot as plt

from other_paper.balancers  import BinaryBalancer, MulticlassBalancer
from other_paper import tools

from fmclp.main import fmclp
from sklearn.model_selection import train_test_split

from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

from fmclp.cuae_metric import cuae

### COMPAS 

In [2]:
compas = pd.read_csv('compas-scores-raw.csv')
                
del compas['Person_ID']
del compas['AssessmentID']
del compas['Case_ID']
del compas['LastName']
del compas['MiddleName']
del compas['FirstName']
del compas['RawScore']
del compas['DecileScore']
del compas['IsCompleted']
del compas['IsDeleted']
del compas['AssessmentReason']
del compas['RecSupervisionLevelText']
del compas['DisplayText']

del compas['Screening_Date']
del compas['DateOfBirth']

def race(x):
    if x == 'African-American':
        return 1
    else:
        return 0
compas['attr']= compas['Ethnic_Code_Text'].apply(race)
del compas['Ethnic_Code_Text']

def targeter(x):
    if x == 'Low':
        return 0
    elif x =='Medium':
        return 1
    else:
        return 2
compas['target'] = compas['ScoreText'].apply(targeter)
del compas['ScoreText']
compas = pd.get_dummies(compas, drop_first = True)

In [3]:
compas

,ScaleSet_ID,RecSupervisionLevel,Scale_ID,attr,target,Agency_Text_DRRD,Agency_Text_PRETRIAL,Agency_Text_Probation,Sex_Code_Text_Male,ScaleSet_Risk and Prescreen,...,CustodyStatus_Prison Inmate,CustodyStatus_Probation,CustodyStatus_Residential Program,MaritalStatus_Married,MaritalStatus_Separated,MaritalStatus_Significant Other,MaritalStatus_Single,MaritalStatus_Unknown,MaritalStatus_Widowed,AssessmentType_New
0,22,1,7,0,0,False,True,False,True,True,...,False,False,False,False,False,False,True,False,False,True
1,22,1,8,0,0,False,True,False,True,True,...,False,False,False,False,False,False,True,False,False,True
2,22,1,18,0,0,False,True,False,True,True,...,False,False,False,False,False,False,True,False,False,True
3,22,1,7,0,0,False,True,False,True,True,...,False,False,False,True,False,False,False,False,False,True
4,22,1,8,0,0,False,True,False,True,True,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60838,22,3,8,1,2,False,False,True,True,True,...,False,False,False,False,False,False,True,False,False,True
60839,22,3,18,1,2,False,False,True,True,True,...,False,False,False,False,False,False,True,False,False,True
60840,22,1,7,0,0,False,True,False,True,True,...,False,False,False,False,False,False,True,False,False,True
60841,22,1,8,0,1,False,True,False,True,True,...,False,False,False,False,False,False,True,False,False,True


In [4]:
s=1
num=100
metric_list = []
accuracy_list = []
diff_list = []
ratio_list = []
variation_list = []

for i in range(num): 
    y = compas.drop('target',axis=1)
    x = compas['target']
    
    y_train,y_test,x_train,x_test = train_test_split(y,x,test_size = 0.3)

    estimator = LGBMClassifier()
    estimator.fit(y_train,x_train)
    estimator_pred= estimator.predict(y_test)
    
    a = y_train['attr'].values
    y = x_train.values
    y_ = np.array(estimator.predict(y_train))
    
    compas_balancer = MulticlassBalancer(y=y, y_=y_, a=a)
    compas_balancer.adjust(goal='odds')  # goal='opportunity'
    compas_preds = compas_balancer.predict(y_ = np.array(estimator.predict(y_test)),a = y_test['attr'].values)
    
    accuracy = accuracy_score(compas_preds, x_test)
    metric = cuae(y_true = x_test, y_pred=compas_preds, sensitive_features=y_test['attr'].values)
    
    accuracy_list.append(accuracy)
    metric_list.append(metric)
    diff_list.append(metric['diff'])
    ratio_list.append(metric['ratio'])
    variation_list.append(metric['variation'])
    #print(accuracy)
    #print(metric)
    print(s)
    s+=1
    
accuracy_list = np.array(accuracy_list)
diff_list = np.array(diff_list)
ratio_list = np.array(ratio_list)
variation_list = np.array(variation_list)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [ ]:
file = open("comparison_results/compas.txt", 'w') 
file.write(f"""accuracy: {accuracy_list}
diff: {diff_list}
ratio: {ratio_list}
var: {variation_list}

accuracy_mean: {accuracy_list.mean()}
diff_mean: {diff_list.mean()}
ratio_mean: {ratio_list.mean()}
var_mean: {variation_list.mean()}""")
file.close()

In [8]:
print(f"""average_accuracy: {accuracy_list[:10].mean()}
average_diff: {diff_list[:10].mean()}
average_ratio: {ratio_list[:10].mean()}
average_variation: {variation_list[:10].mean()}""")

average_accuracy: 0.7293266860242152
average_diff: 0.0787752689697261
average_ratio: 1.185371650200936
average_variation: 0.7686252585085858
